In [1]:
!pip install -U tfx

In [12]:
import os
import tempfile
import urllib
from tfx import v1 as tfx
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datatkpxe9sm/data.csv', <http.client.HTTPMessage at 0x7f507fd9bf50>)

In [13]:
from tfx.proto import example_gen_pb2
# 학습, 평가, 테스트 데이터셋을 각각 6:2:2의 비율로 분할 / 분할 비율은 hash_buckets이 담당 / 기본 분할은 2:1 (학습, 평가)
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
                                                     example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
                                                     example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2),
    ])
)

In [15]:
# 파이프라인 컴포넌트를 인스턴스화합니다.
example_gen = CsvExampleGen(input_base=_data_root, output_config=output)
# 대화식으로 컴포넌트를 실행합니다.
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [10]:
for artifact in example_gen.outputs['examples'].get():
  print(artifact)

Artifact(artifact: id: 3
type_id: 14
uri: "/tmp/tfx-interactive-2022-05-18T14_15_03.822340-645nf5vf/CsvExampleGen/examples/4"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1652883304,sum_checksum:1652883304"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.7.1"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "v

In [ ]:
# 기존 하위 디렉터리에 나눠진 데이터를 가져올 수 있다.
# input = example_gen_pb2.Input(splits=[
#     example_gen_pb2.Input.Split(name='train', pattern='train/*'),
#     example_gen_pb2.Input.Split(name='eval', pattern='eval/*'),
#     example_gen_pb2.Input.Split(name='test', pattern='test/*')
# ])


In [ ]:
# 일정 기간마다 ETL 프로세스가 새로운 데이터 스냅샷을 만들어 새 SPAN을 생성 / SPAN 번호는 "메타데이터"에 저장
# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(name='train',
#                                             pattern='span-{SPAN}/train/*'),
#                 example_gen_pb2.Input.Split(name='eval',
#                                             pattern='span-{SPAN}/eval/*')
#             ])

TFX ExampleGen 컴포넌트에서는 현재 데이터셋 버전관리를 지원하고 있지 않기 때문에 외부 툴을 사용해야함

* DVC
* 파키덤

